## Create Sampled Dataset of KITTI


In [26]:
import argparse
import os
import yaml
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from matplotlib import cm
from functools import partial

from sphere import Sphere
from dh_grid import DHGrid
from laserscan import SemLaserScan
from visualize import Visualize

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
def load_sequence(dataroot, sequence):
    scan_paths = f'{dataroot}/{sequence}/velodyne'
    scan_names = [os.path.join(dp, f) for dp, dn, fn in os.walk(
      os.path.expanduser(scan_paths)) for f in fn]
    scan_names.sort()
    
    label_paths = f'{dataroot}/{sequence}/labels'
    label_names = [os.path.join(dp, f) for dp, dn, fn in os.walk(
        os.path.expanduser(label_paths)) for f in fn]
    label_names.sort()    
    assert len(label_names) == len(scan_names)
    print(f'Found {len(scan_names)} pointclouds and labels for sequence {sequence}.')
    return scan_names, label_names

def load_config_file(config_file):    
    try:        
        CFG = yaml.safe_load(open(config_file, 'r'))        
        return CFG
    except Exception as e:
        print(e)        
        return None

# dataroot = '/mnt/data/datasets/KITTI/sequences'
dataroot = '/media/berlukas/Data/data/datasets/KITTI/sequences'
sequences = ['04']
config_file = '../config/semantic-kitti.yaml'

print(f'Setting dataroot to {dataroot}.')
print(f'Configured {len(sequences)} sequences.')
print(f'Configured config file {config_file}')

Setting dataroot to /media/berlukas/Data/data/datasets/KITTI/sequences.
Configured 1 sequences.
Configured config file ../config/semantic-kitti.yaml


In [47]:
def progresser(sample, grid, auto_position=True, write_safe=False, blocking=True, progress=False):    
    sample_sphere = Sphere(sample)
    return sample_sphere.sampleUsingGrid(grid)

CFG = load_config_file(config_file)
color_dict = CFG["color_map"]
nclasses = len(color_dict)
scan = SemLaserScan(nclasses, color_dict, project=False)
bw = 100
assert CFG is not None
  
sem_features = []
export_ds = '/media/berlukas/Data/data/datasets/KITTI/processed'
for seq in sequences:
    print(f'Loading sequence {seq}.')
    scan_names, label_names = load_sequence(dataroot, seq)
    n_scans = len(scan_names)
    n_scans = 5
    all_sem_clouds = []    
    for i in tqdm(range(0, n_scans)):
        scan.open_scan(scan_names[i])
        scan.open_label(label_names[i])
        scan.colorize()
        
        pc = np.column_stack((scan.points, scan.remissions, scan.sem_label, scan.inst_label))
        mask = pc[:,4] > 0
        all_sem_clouds.append(pc[mask])         
        
    print(f"Loading complete. Computing features...")    
    grid, _ = DHGrid.CreateGrid(bw)
    
    # parallel
#     sem_features = process_map(partial(progresser, grid=grid), all_sem_clouds, max_workers=8, chunksize=100)    
    
    # seq    
    for cloud in tqdm(all_sem_clouds):
        cur_features = progresser(cloud, grid)
        sem_features.append(cur_features)    
        
filename = f"{export_ds}/clouds.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}.")

Loading sequence 04.
Found 271 pointclouds and labels for sequence 04.



Loading complete. Computing features...


Input data has semantics.
Input data has semantics.
Input data has semantics.
Input data has semantics.
Input data has semantics.

Wrote features to /media/berlukas/Data/data/datasets/KITTI/processed/clouds.npy.


In [40]:
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def visualizeRawPointcloud(pcl, val):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl[:, 0:3])
    colors = mapIntensityToRGB(val)
#     colors = scan.sem_color_lut[pcl[:,4].astype(np.int)]
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.visualization.draw_geometries([pcd])    
    
pc = all_sem_clouds[3]
visualizeRawPointcloud(pc, pc[:, 5])

In [ ]:
def createGrid_old(bw):
        n_grid = 2 * bw
        k = 0;
        points = np.empty([n_grid * n_grid, 2])
        for i in range(n_grid):
            for j in range(n_grid):
                points[k, 0] = (np.pi*(2*i+1))/(4*bw)
                points[k, 1] = (2*np.pi*j)/(2*bw);
                k = k + 1;
        return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])    
    return cart_grid

def create_sampling_sphere(bw):
    grid = createGrid_old(bw)
    xyz_grid = convertGridToEuclidean_old(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid.T

cur_sem_cloud = sem_features[0]
cur_sem_cloud = np.reshape(cur_sem_cloud, (4, -1)).T
print(f'{cur_sem_cloud.shape}')
pc = create_sampling_sphere(bw)
points_xyz = pc.T[:,0:3]
print(f"sampling pointcloud shape is {points_xyz.shape}")
print(f"feature shape is {cur_sem_cloud.shape}")
points_xyzl = np.column_stack((points_xyz, cur_sem_cloud[:,2]))
# points_xyzn = np.column_stack((points_xyz, cur_sem_cloud[:,2]))


visualizeRawPointcloud(points_xyzl, points_xyzl[:, 3])